# Виконати детекцію границь на зображеннях за допомогою операторів:
# * Roberts
# * Sobel
# Провести порівняльний аналіз.

In [71]:
import os
import cv2
import pickle
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

from cv.helpers import *

from IPython.core.display_functions import clear_output

x_sobel = np.array([
    [-1, 0, 1],
    [-2, 0, 2],
    [-1, 0, 1]
], dtype=np.float32)
y_sobel = np.array([
    [-1, -2, -1],
    [0, 0, 0],
    [1, 2, 1]
], dtype=np.float32)

x_roberts = np.array([
    [1, 0],
    [0, -1]
], dtype=np.float32)
y_roberts = np.array([
    [0, -1],
    [1, 0]
], dtype=np.float32)

def point_operation(img, K, L):
    img = img * K + L
    img[img > 255] = 255
    img[img < 0] = 0
    return np.asarray(img, dtype=np.uint8)

In [72]:
im = cv2.imread('src/im1.png')
gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
show(im)

113

In [73]:
# noise add
noise = cv2.randu(np.zeros(gray.shape), 0, 255)
noisy_gray = gray + np.array(0.5 * noise, dtype=np.float32)
show_2(gray, noisy_gray)

# point operation
show_2(gray, point_operation(gray, 0.5, 125))

# blur effect
size = 3
blur = np.ones((size, size), np.float32) / size ** 2
show_2(gray, cv2.filter2D(gray, -1, blur))

# lib filters
show_2(cv2.filter2D(gray, -1, x_sobel), cv2.filter2D(gray, -1, y_sobel))
show_2(cv2.filter2D(gray, -1, x_roberts), cv2.filter2D(gray, -1, y_roberts))

113

In [74]:
def get_surround_mtx(img, krn, i, j, inter):
    rows, cols = img.shape[:2]
    k_rows, k_cols = krn.shape[:2]

    mtx = np.ndarray((k_rows, k_cols), dtype=np.float32)

    for k in range(k_rows):
        for l in range(k_cols):
            k_i, l_i = k - 1 + i, l - 1 + j # step back values

            if 0 > k_i or k + i > rows or 0 > l_i or l + j > cols:  # out of bounds
                if inter:
                    mtx[k, l] = img[i, j]  # fake interpolation
                else:
                    mtx[k, l] = 0  # no interpolation
            else:
                mtx[k, l] = img[k_i, l_i]

    return mtx


def apply_kernel(img, krn, inter=False):
    if len(img.shape) == 3:
        rows, cols, dim = img.shape
        new_img = np.zeros((rows, cols, dim), dtype=np.float32)
    else:
        (rows, cols), dim = img.shape, 0
        new_img = np.zeros((rows, cols), dtype=np.float32)

    for i in tqdm(range(rows)):
        for j in range(cols):
            if dim == 0:
                mtx = get_surround_mtx(img, krn, i, j, inter)
                new_img[i, j] = (mtx * krn).sum()
            else:
                for d in range(dim):
                    tm_im = img[:,:,d]  # get 1 of colors R / G / B
                    mtx = get_surround_mtx(tm_im, krn, i, j, inter)
                    new_img[i, j, d] = (mtx * krn).sum()

            new_img[new_img > 255] = 255
            new_img[new_img < 0] = 0
    return np.asarray(new_img, dtype=np.float32)


def process(pth, kernel, ratio=0, interpolate=False, grayscale=False):
    img = cv2.imread(pth)

    if ratio != 0:
        img = cv2.resize(img, (0, 0), fx=ratio, fy=ratio)

    if grayscale:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    my = apply_kernel(img, kernel, interpolate)
    lib = cv2.filter2D(img, -1, kernel)

    return my, lib

In [75]:
im_pths = ['src/im1.png', 'src/im2.png', 'src/im3.png', 'src/im4.png', 'src/im5.png']
im_kernels = [(x_sobel, 'x_sobel'), (y_sobel, 'y_sobel'), (x_roberts, 'x_roberts'), (y_roberts, 'y_roberts')]

if os.path.exists('results.pkl'):
    results = pickle.load(open("results.pkl", 'rb'))
else:
    results = []
    for im_pth in im_pths:
        print(im_pth, end=' ')
        for im_kernel, k_name in im_kernels:
            results.append((process(im_pth, im_kernel, ratio=0.6, grayscale=True), k_name))
    pickle.dump(results, open("results.pkl", "wb"))

In [78]:
def iter_view(res):
    idx = 0
    while True:
        imgs, name = res[idx]
        key = show_2(*imgs, name=name)

        if key & 0xFF == ord('a'):
            if idx == 0:
                idx = len(results) - 1
            else:
                idx -= 1
        elif key & 0xFF == ord('d'):
            if idx == len(results) - 1:
                idx = 0
            else:
                idx += 1
        elif key & 0xFF == ord('q'):
            break

iter_view(results)

In [79]:
path = 'src/im5.png'
kernel = x_roberts

size = 3
blur = np.ones((size, size), np.float32) / size ** 2

no_int = process(path, kernel, ratio=0.3, interpolate=True)[0]
interp = process(path, kernel, ratio=0.3, interpolate=False)[0]

show_2(scale(no_int), scale(interp))

100%|██████████| 191/191 [00:02<00:00, 66.11it/s]


0